# CSR: Compressed Sparse Row für dünnbesetzte Matrizen

In [2]:
import numpy as np

In [3]:
matrix = np.array([[1, 4, 0, 0, 1],
                    [0, 0, 0, 0, 0],
                    [0, 5, 6, 0, 0],
                    [0, 0, 3, 0, 0]])
matrix


array([[1, 4, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 5, 6, 0, 0],
       [0, 0, 3, 0, 0]])

In [4]:
vector = np.array([3, 4, 5, 6, 7])
vector

array([3, 4, 5, 6, 7])

In [5]:
# Matrix-Vector-Multiply:
# For each row in matrix: Multiply pairwise elements with vector
matrix @ vector

array([26,  0, 50, 15])

In [6]:
# CSR-Representation
# val: Non-zero values in Reihenfolge der Matrix (zeilenweise)
val = np.array([1, 4, 1, 5, 6, 3])
# col_index: in welcher Spalte steht der jeweilige Wert?
col_index = np.array([0, 1, 4, 1, 2, 2])
# row_ptr: bei welchen Index startet die Zeile? 
# Wenn nur Nullen in einer Zeile, dann Index der nächsten Zeile
# Zusätzliches Element mit Anzahl der Non-Zero Values
row_ptr = np.array([0, 3, 3, 5, 6])

In [7]:
# Result contains one row per row in the matrix (first dim of matrix, shape[0])
result = np.zeros(matrix.shape[0])
for row in range(matrix.shape[0]):
    # In each row we do pairwise elements multiplication
    # what are all the entries belonging to one row?
    # We start at row_ptr[row] and end before beginning of next row
    for ptr in range(row_ptr[row], row_ptr[row+1]):
        # To know which position in the vector to take, we look at col_index
        result[row] += val[ptr] * vector[col_index[ptr]]
result



array([26.,  0., 50., 15.])

In [12]:
def to_csr(matrix):
    val = []
    col_index = []
    row_ptr = []
    n_non_zero = 0

    for y in range(matrix.shape[0]):
        non_zero_in_row = False
        for x in range(matrix.shape[1]):
            if matrix[y,x] != 0:
                val.append(matrix[y,x])
                col_index.append(x)
                if not non_zero_in_row:
                    row_ptr.append(n_non_zero)
                    non_zero_in_row = True
                n_non_zero += 1
        if not non_zero_in_row:
            row_ptr.append(-1)
    for y in range(matrix.shape[0]-1, -1, -1):
        print(y)
        if row_ptr[y] == -1:
            row_ptr[y] = row_ptr[y+1]

    row_ptr.append(n_non_zero)

    return val, col_index, row_ptr
        


In [13]:
to_csr(matrix)

3
2
1
0


([1, 4, 1, 5, 6, 3], [0, 1, 4, 1, 2, 2], [0, 3, 3, 5, 6])

In [49]:
def to_csr_np(matrix):
    val = matrix[matrix.nonzero()]
    col_index = matrix.nonzero()[1]
    row_ptr = np.concatenate([[0], np.apply_along_axis(np.sum, 1, matrix > 0).cumsum()])
    return val, col_index, row_ptr
    

In [50]:
to_csr_np(matrix)

(array([1, 4, 1, 5, 6, 3]), array([0, 1, 4, 1, 2, 2]), array([0, 3, 3, 5, 6]))